# Нейросетевой анализ

Мы приведём два варианта нейросетевого анализа набора данных. Первый заключается в построении полносвязной нейронной сети по изображениям, *растянутым* в вектора. Второй вариант состоит в построении свёрточной нейронной сети. 

Импортируем инструменты

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

Импортируем данные

In [ ]:
data = pd.read_parquet("images.parquet")
data.info()
data.head()

Составляем матрицу объекты-признаки и вектор целевой переменной

In [ ]:
X = data.drop(columns=["target"]).values
y = np.array(data["target"].values.reshape(-1, 1), dtype=np.int32)
X.shape, y.shape

Разбиваем выборку на обучающую и тестовую 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Стандартизируем данные

In [ ]:
ss = StandardScaler().fit(X_train)

X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

Определяем модель

In [ ]:
model = Sequential(name="full_connected")

model.add(Input(shape=(X_train_scaled.shape[1],)))
model.add(Dense(1500, name="color", dtype=np.float32)) 
model.add(Dropout(0.33))
model.add(Dense(300, activation="relu", dtype=np.float64))
model.add(Dense(60, activation="relu", dtype=np.float64))
model.add(Dense(12, activation="relu", dtype=np.float64))
model.add(Dense(1, activation="sigmoid", name="output_layer", dtype=np.float64))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Проводим обучение

In [ ]:
model.fit(X_train_scaled, y_train, epochs=4, batch_size=32)

Проверяем качество

In [ ]:
y_pred = model.predict(X_test_scaled).flatten()

In [ ]:
accuracy_score(y_test.flatten(), np.array(y_pred > 0.5, dtype=np.int16))

In [ ]:
roc_auc_score(y_test.flatten(), y_pred)